# Forecasting Ontario's Energy Demand with Climate and Population Data  
## Data Collection and Data Preprocessing Overview

### Dataset Description

<table style="margin-left: 0;">
  <tr>
    <th>Dataset</th>
    <th>Description</th>
    <th>Source</th>
  </tr>
  <tr>
    <td>Energy Demand</td>
    <td>Hourly electricity demand and price</td>
    <td><a href="https://www.kaggle.com/code/pythonafroz/ontario-energy-demand-forecast-with-time-series/input">Energy Demand Forecast</a></td>
  </tr>
  <tr>
    <td>Weather Data</td>
    <td>Historical weather features from 10 stations</td>
    <td><a href="https://climate.weather.gc.ca/historical_data/search_historic_data_stations_e.html?searchType=stnProv&timeframe=1&lstProvince=ON&optLimit=yearRange&StartYear=2003&EndYear=2023&Year=2025&Month=2&Day=17&selRowPerPage=25">Gov. of Canada – Ontario</a></td>
  </tr>
  <tr>
    <td>Population</td>
    <td>Quarterly population estimates</td>
    <td><a href="https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1710000901&cubeTimeFrame.startMonth=01&cubeTimeFrame.startYear=2003&cubeTimeFrame.endMonth=10&cubeTimeFrame.endYear=2023&referencePeriods=20030101%2C20231001">Statistics Canada</a></td>
  </tr>
</table>

---

### Dataset Collection

<table style="margin-left: 0;">
  <tr>
    <th>Dataset</th>
    <th>Method</th>
    <th>Tool</th>
  </tr>
  <tr>
    <td>Energy Demand</td>
    <td>CSV Download</td>
    <td>Manual</td>
  </tr>
  <tr>
    <td>Weather Data</td>
    <td>Web Scraping</td>
    <td>Selenium</td>
  </tr>
  <tr>
    <td>Population</td>
    <td>Web Scraping</td>
    <td>Python</td>
  </tr>
</table>

---

### Dataset Preprocessing

<table style="margin-left: 0;">
  <tr>
    <th>Dataset</th>
    <th>Preprocessing Steps</th>
  </tr>
  <tr>
    <td>Energy Demand</td>
    <td>Check for NaNs, sample-and-hold</td>
  </tr>
  <tr>
    <td>Weather Data</td>
    <td>Drop duplicates, fill missing, circular mean for wind, average across stations</td>
  </tr>
  <tr>
    <td>Population</td>
    <td>Check for NaNs, linear interpolation</td>
  </tr>
</table>


# Ontario Weather Dataset Web Scraping

In [ ]:
# ------------------------------
# Environment Setup for Selenium with Google Chrome
# ------------------------------

# Update package list and install Google Chrome
!apt update -y
!apt install -y google-chrome-stable

# Install Python package for managing WebDriver binaries
!pip install webdriver-manager

# Extract the installed version of Google Chrome (e.g., 121.x.x.x → 121)
CHROME_VERSION = !google-chrome --version
CHROME_VERSION = CHROME_VERSION[0].split()[2].split('.')[0]

# Download and install the matching version of ChromeDriver
!wget -q "https://chromedriver.storage.googleapis.com/$(curl -sS https://chromedriver.storage.googleapis.com/LATEST_RELEASE_$CHROME_VERSION)/chromedriver-linux64.zip"
!unzip -o chromedriver-linux64.zip -d /usr/local/bin/
!chmod +x /usr/local/bin/chromedriver

# Confirm that ChromeDriver was installed correctly
!chromedriver --version

# Install the Selenium library
!pip install selenium

# ------------------------------
# Selenium WebDriver Configuration
# ------------------------------

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Define paths to the Chrome binary and ChromeDriver
chrome_path = "/usr/bin/google-chrome"
chromedriver_path = "/usr/local/bin/chromedriver"

# Configure Chrome options
options = Options()
options.binary_location = chrome_path
# options.add_argument("--headless")  # Uncomment to run Chrome in headless mode
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Set up the ChromeDriver service
service = Service(chromedriver_path)

# Example usage (uncomment to use):
# driver = webdriver.Chrome(service=service, options=options)

# ------------------------------
# Alternative Setup (Full Keyring + Repository Installation)
# ------------------------------

# Update system and install dependencies for secure package handling
!apt-get update
!apt-get install -y wget unzip

# Add Google Chrome’s signing key and repository manually
!wget -q -O - https://dl.google.com/linux/linux_signing_key.pub | gpg --dearmor > /usr/share/keyrings/google-chrome-keyring.gpg
!echo 'deb [signed-by=/usr/share/keyrings/google-chrome-keyring.gpg] http://dl.google.com/linux/chrome/deb/ stable main' | tee /etc/apt/sources.list.d/google-chrome.list

# Install Google Chrome (again, from newly added repo)
!apt-get update
!apt-get install -y google-chrome-stable

# Download the latest ChromeDriver release (general version)
!wget https://chromedriver.storage.googleapis.com/$(curl -sS https://chromedriver.storage.googleapis.com/LATEST_RELEASE)/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!mv chromedriver /usr/local/bin/
!chmod +x /usr/local/bin/chromedriver

In [ ]:
import time
import traceback
import os
import zipfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def get_chrome_driver():
    """
    Configures and returns a Chrome WebDriver instance with predefined options.
    Sets the default download directory and disables GPU/Dev Shm for compatibility.
    """
    options = webdriver.ChromeOptions()
    
    # Headless mode is disabled for debugging; uncomment if needed
    # options.add_argument("--headless=new")
    
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")

    # Define Chrome's default download behavior
    download_dir = os.getcwd()
    prefs = {
        "download.default_directory": download_dir,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
    }
    options.add_experimental_option("prefs", prefs)

    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def wait_for_download(directory, timeout=30):
    """
    Waits for all Chrome .crdownload files to complete download within a given timeout.
    """
    time_waited = 0
    while any(fname.endswith(".crdownload") for fname in os.listdir(directory)):
        if time_waited > timeout:
            print("Warning: Timeout while waiting for file download.")
            break
        time.sleep(1)
        time_waited += 1

def download_data_for_url(url, station_name):
    """
    Automates download of monthly weather data from a specified URL and station name.
    Downloads all available months between 2003 and 2023 and compresses the result.
    """
    driver = None
    try:
        station_dir = os.path.join(os.getcwd(), station_name)
        os.makedirs(station_dir, exist_ok=True)

        driver = get_chrome_driver()

        # Allow downloads via Chrome DevTools Protocol
        driver.command_executor._commands["send_command"] = ("POST", "/session/$sessionId/chromium/send_command")
        params = {
            "cmd": "Page.setDownloadBehavior",
            "params": {"behavior": "allow", "downloadPath": station_dir},
        }
        driver.execute("send_command", params)

        wait = WebDriverWait(driver, 40)
        driver.get(url)
        time.sleep(5)  # Allow full page load

        # Optional: Debug info for <select> elements and iframes
        select_elements = driver.find_elements(By.TAG_NAME, "select")
        print(f"Found {len(select_elements)} <select> elements.")
        for idx, select in enumerate(select_elements):
            print(f"Select {idx}: {select.get_attribute('outerHTML')}")

        iframes = driver.find_elements(By.TAG_NAME, "iframe")
        print(f"Found {len(iframes)} iframes.")
        if iframes:
            driver.switch_to.frame(iframes[0])
            print("Switched to first iframe.")

        # Locate and interact with the year dropdown
        year_container = wait.until(EC.presence_of_element_located((By.XPATH, "//select[contains(@id, 'Year')]")))
        year_select = Select(year_container)
        valid_years = [y.text.strip() for y in year_select.options if 2003 <= int(y.text.strip()) <= 2023]
        print(f"Years to process: {valid_years}")

        for year_text in valid_years:
            year_container = wait.until(EC.presence_of_element_located((By.XPATH, "//select[contains(@id, 'Year')]")))
            year_select = Select(year_container)
            year_select.select_by_visible_text(year_text)

            for month_index in range(12):  # Loop through all months
                try:
                    month_container = wait.until(EC.presence_of_element_located((By.XPATH, "//select[contains(@id, 'Month')]")))
                    month_select = Select(month_container)
                    month_option = month_select.options[month_index]
                    month_text = month_option.text.strip()

                    month_select.select_by_visible_text(month_text)
                    print(f"Processing: {year_text} - {month_text}")

                    # Click "Go" to load data
                    go_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@type='submit' and contains(@class, 'btn-primary')]")))
                    driver.execute_script("arguments[0].scrollIntoView(true);", go_button)
                    go_button.click()

                    # Click "Download Data"
                    download_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@type='submit' and @value='Download Data']")))
                    download_button.click()

                    # Wait until the download is complete
                    wait_for_download(station_dir)

                except Exception as e:
                    print(f"Error during download for {year_text} - {month_text}: {e}")
                    continue  # Continue with next month

        # Create a ZIP archive of the station's downloaded CSVs
        zip_path = os.path.join(os.getcwd(), f"{station_name}_downloads.zip")
        with zipfile.ZipFile(zip_path, "w") as zipf:
            for file in os.listdir(station_dir):
                file_path = os.path.join(station_dir, file)
                if file.endswith(".csv"):
                    zipf.write(file_path, os.path.basename(file_path))

        print(f"Data for {station_name} compressed to {zip_path}")

    except Exception as e:
        print(f"Download failed for {station_name}: {e}")
        traceback.print_exc()

    finally:
        if driver:
            print("Closing WebDriver...")
            driver.quit()
            print("WebDriver closed.")

# Update the stations here, for example Guelph
stations = {
    "Guelph": "https://climate.weather.gc.ca/climate_data/hourly_data_e.html?hlyRange=2006-12-15%7C2022-04-12&dlyRange=2006-09-28%7C2022-04-12&mlyRange=2006-12-01%7C2006-12-01&StationID=45407&Prov=ON&urlExtension=_e.html&searchType=stnProv&optLimit=yearRange&StartYear=2003&EndYear=2023&selRowPerPage=25&Line=126&Month=4&Day=12&lstProvince=ON&timeframe=1&Year=2022"
}

# Loop through each station and trigger download
for station_name, url in stations.items():
    print(f"\nStarting download for: {station_name}")
    download_data_for_url(url, station_name)


In [ ]:
# Make sure that you are inside the folder
import os
print(os.getcwd())
os.chdir("fill-missing-data")  # Change to the inner folder
print(os.getcwd())  # Verify the new working directory

# Ontario Weather Dataset Imputation and Processing


This script performs the following tasks:  

### 1. **Set the Working Directory**  
- Ensures the script runs from the correct folder (**`fill-missing-data`**).  
- Uses `os.chdir()` to change to the correct working directory.  
- Verifies the new directory with `os.getcwd()`.  

### 2. **Install Required Libraries**  
- Installs `pandas` (for data manipulation) and `scipy` (for statistical calculations).  
- Uses `!pip install pandas scipy` to ensure dependencies are available.  

### 3. **Handle Missing Weather Data**  
- Uses `impute_missing_data(df)` to clean and fill missing values.  
- Applies **linear interpolation** for temperature, dew point, and wind speed.  
- Uses **forward-fill and backward-fill** for relative humidity and station pressure.  
- Imputes missing wind direction using **circular mean interpolation**.  
- Ensures **realistic temperature and wind speed ranges** to prevent outliers.  
- Replaces zeros in key meteorological columns with `NaN` and fills them with appropriate values.  

### 4. **Process and Save Cleaned Data**  
- `process_all_stations(file_paths, output_dir)` reads multiple datasets.  
- The function imputes missing values and saves the cleaned data into **`../imputed_datasets`**.  

### 5. **Example Usage**  
- The script processes a list of CSV files, including **`Consolidated_Toronto.csv`**.  
- The output files are saved with imputed values in the specified directory.  

### Note  
- Ensure the script is executed in the correct folder (**`fill-missing-data`**).  
- All CSV files should exist in the working directory before running.  
- The imputation logic prevents unrealistic weather conditions from affecting the dataset.  

### Workflow  
- `impute_missing_data(df)`: Cleans missing values using appropriate interpolation techniques.  
- `process_all_stations(file_paths, output_dir)`: Processes multiple station datasets and saves the cleaned versions.  

In [ ]:
# Install required libraries (pandas for data manipulation, scipy for statistical and interpolation functions)
!pip install pandas scipy

### `impute_missing_data(df)`  
**Purpose:** Handles missing weather data imputation.  

**How it Works:**  
- Uses **linear interpolation** for temperature, dew point, and wind speed.  
- Applies **forward-fill and backward-fill** for relative humidity and station pressure.  
- Uses **circular mean interpolation** for wind direction.  
- Ensures **realistic temperature and wind speed ranges** to prevent outliers.  
- Replaces zeros in key meteorological columns with `NaN` and fills them appropriately.  

**What to Look For:**  
- Missing values should be interpolated smoothly without unrealistic jumps.  
- Ensure imputed values align with expected weather trends.  


In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from scipy.stats import circmean
import os

def impute_missing_data(df):
    # Identify existing columns for dropping metadata rows
    # Ensure at least some valid values exist before interpolation
    # This step handles missing values for temperature, dew point temperature, and wind speed
    # by using linear interpolation. The method helps fill in gaps in a time series dataset,
    # ensuring smoother transitions and preventing data loss.
    required_columns = ['Temp (°C)', 'Dew Point Temp (°C)', 'Wind Spd (km/h)', 'Rel Hum (%)', 'Stn Press (kPa)', 'Visibility (km)', 'Wind Dir (10s deg)', 'Weather']
    existing_columns = [col for col in required_columns if col in df.columns]

    # Remove rows that only contain metadata (no weather-related data)
    if existing_columns:
        df = df.dropna(how='all', subset=existing_columns)

    # Ensure at least some valid values exist before interpolation
    for col in ['Temp (°C)', 'Dew Point Temp (°C)', 'Wind Spd (km/h)']:
        if col in df.columns and df[col].notna().sum() > 1:
            df.loc[:, col] = df[col].interpolate(method='linear', limit_direction='both')
            df.loc[:, col] = df[col].bfill().ffill().round(2)

    # Ensure temperature is within a realistic seasonal range (-40 to 40°C)
    # This prevents unrealistic temperature readings, which could arise due to faulty sensors or data errors.
    if 'Temp (°C)' in df.columns:
        df.loc[:, 'Temp (°C)'] = df['Temp (°C)'].clip(lower=-40, upper=40)

    # Ensure wind speed is within a reasonable range (1 to 100 km/h)
    # Wind speed should be non-negative and within a typical range to prevent incorrect calculations.
    if 'Wind Spd (km/h)' in df.columns:
        df.loc[:, 'Wind Spd (km/h)'] = df['Wind Spd (km/h)'].clip(lower=1, upper=100)
        df.loc[:, 'Wind Spd (km/h)'] = df['Wind Spd (km/h)'].fillna(df['Wind Spd (km/h)'].median()).round(2)

    # Forward and Backward Fill for Relative Humidity and Station Pressure
    # These meteorological variables should not have zero values, so we replace zeros with NaN and use
    # forward and backward filling to ensure continuity in the dataset.
    for col in ['Rel Hum (%)', 'Stn Press (kPa)']:
        if col in df.columns:
            df.loc[:, col] = df[col].replace(0, np.nan)
            df.loc[:, col] = df[col].bfill().ffill().round(2)

    # Circular Mean Imputation for Wind Direction
    # Wind direction is a circular variable (0° and 360° are equivalent).
    # We convert it to radians, interpolate missing values, and convert it back to degrees to maintain accuracy.
    if 'Wind Dir (10s deg)' in df.columns:
        df.loc[:, 'Wind Dir (10s deg)'] = df['Wind Dir (10s deg)'].replace(0, np.nan)
        df.loc[:, 'Wind Dir (10s deg)'] = df['Wind Dir (10s deg)'].apply(lambda x: np.deg2rad(x) if pd.notnull(x) else np.nan)
        df.loc[:, 'Wind Dir (10s deg)'] = df['Wind Dir (10s deg)'].interpolate()
        df.loc[:, 'Wind Dir (10s deg)'] = df['Wind Dir (10s deg)'].apply(lambda x: np.rad2deg(x)).round(2)

    # Location-Based Mean for Visibility
    # Visibility values should not be zero. We replace zero with NaN and fill missing values using the median,
    # which is a robust method for handling outliers.
    if 'Visibility (km)' in df.columns:
        df.loc[:, 'Visibility (km)'] = df['Visibility (km)'].replace(0, np.nan)
        df.loc[:, 'Visibility (km)'] = df['Visibility (km)'].fillna(df['Visibility (km)'].median()).round(2)

    # Wind Chill Calculation (Fixing large negative values issue)
    # Wind chill is calculated when temperature and wind speed are available.
    # This formula helps estimate how cold it feels due to wind, preventing unrealistic extreme values.
    if 'Wind Chill' in df.columns and 'Temp (°C)' in df.columns and 'Wind Spd (km/h)' in df.columns:
        mask = df['Wind Chill'].isna()
        valid_wind_mask = df['Wind Spd (km/h)'] > 0
        valid_mask = mask & valid_wind_mask
        df.loc[valid_mask, 'Wind Chill'] = 13.12 + 0.6215 * df.loc[valid_mask, 'Temp (°C)'] - 11.37 * (df.loc[valid_mask, 'Wind Spd (km/h)'] ** 0.16) + 0.3965 * df.loc[valid_mask, 'Temp (°C)'] * (df.loc[valid_mask, 'Wind Spd (km/h)'] ** 0.16)
        df.loc[:, 'Wind Chill'] = df['Wind Chill'].bfill().ffill().round(2)

    # Mode Imputation for Weather Condition
    # Since weather conditions are categorical, we use mode (most frequently occurring value) to fill missing values.
    # If mode() is empty (all values missing), we default to 'Unknown'.
    if 'Weather' in df.columns:
        most_frequent_weather = df['Weather'].mode()[0] if not df['Weather'].mode().empty else 'Unknown'
        df.loc[:, 'Weather'] = df['Weather'].fillna(most_frequent_weather)

    return df

def process_all_stations(file_paths, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for file in file_paths:
        df = pd.read_csv(file)
        df_imputed = impute_missing_data(df)
        output_file = os.path.join(output_dir, os.path.basename(file))
        df_imputed.to_csv(output_file, index=False)

    print(f"Imputed data saved to: {output_dir}")

# Example usage:
file_paths = [
    # "Consolidated_Algonquin.csv",
    # "Consolidated_Guelph.csv",
    # "Consolidated_KitchenerWaterloo.csv",
    # "Consolidated_LakeSuperiorProvincialPark.csv",
    # "Consolidated_London.csv",
    #"Consolidated_NorthernOntario.csv",
    # "Consolidated_Ottawa.csv",
    # "Consolidated_ThunderBay.csv",
    # "Consolidated_Toronto.csv",
    # "Consolidated_Windsor.csv"
    "Consolidated_Toronto.csv"
]

process_all_stations(file_paths, "../imputed_datasets")


/var/folders/1b/gz7s8qf92rg37q344skx3cmw0000gn/T/ipykernel_96446/853362762.py:85: DtypeWarning: Columns (10,12,14,16,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/opt/anaconda3/envs/ELEC390_Lab1/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/var/folders/1b/gz7s8qf92rg37q344skx3cmw0000gn/T/ipykernel_96446/853362762.py:76: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Unknown' 'Unknown' 'Unknown' ... 'Unknown' 'Unknown' 'Unknown']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, 'Weather'] = df['Weather'].fillna(most_frequent_weather)


Imputed data saved to: ../imputed_datasets


In [ ]:
import pandas as pd

# Load the electricity demand dataset (CSV format)
df_csv = pd.read_csv("Ontario_Electricity_Demand_with_Population.csv")
print("CSV data loaded successfully.")
display(df_csv.head())

# Load the consolidated dataset (Excel format)
df_excel = pd.read_excel("Consolidated_Ontario_Final (1).xlsx")
print("Excel data loaded successfully.")
display(df_excel.head())

# Identify common columns between the two datasets for merging
common_columns = list(set(df_csv.columns) & set(df_excel.columns))
print("Common columns identified for merge:", common_columns)

# Proceed to merge only if common columns are found
if common_columns:
    # Perform an inner merge on the first common column found
    df_merged = pd.merge(df_csv, df_excel, on=common_columns[0], how='inner')
    print("Datasets merged successfully.")
    display(df_merged.head())

    # Sort the merged data by date column if present
    date_column = 'Date'  # Update if your actual date column uses a different name
    if date_column in df_merged.columns:
        df_merged[date_column] = pd.to_datetime(df_merged[date_column])
        df_merged = df_merged.sort_values(by=date_column)
        print(f"Merged data sorted by '{date_column}'.")
        display(df_merged.head())
    else:
        print("Note: No 'Date' column found for sorting.")
else:
    print("Warning: No common columns found for merging.")

# Ontario Population Dataset Preprocess

In [ ]:
# Install required packages
!pip install selenium pandas openpyxl

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import time

# --------------------------------------------------------
# Initialize Headless Chrome WebDriver
# --------------------------------------------------------
# Ensure that ChromeDriver is installed and available in your system PATH
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run Chrome in headless mode (no GUI)
driver = webdriver.Chrome(options=options)

# --------------------------------------------------------
# Navigate to Statistics Canada - Ontario Population Table
# --------------------------------------------------------
url = (
    "https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?"
    "pid=1710000901&cubeTimeFrame.startMonth=01&cubeTimeFrame.startYear=2003&"
    "cubeTimeFrame.endMonth=10&cubeTimeFrame.endYear=2023&"
    "referencePeriods=20030101%2C20231001"
)
driver.get(url)

# Allow time for the interactive table to fully render
time.sleep(5)

# --------------------------------------------------------
# Scrape Table Content for Ontario
# --------------------------------------------------------
table = driver.find_element(By.CLASS_NAME, "T1__table")
rows = table.find_elements(By.TAG_NAME, "tr")

ontario_data = []
headers = []

for row in rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    if not cells:
        # Extract column headers from the header row
        headers = [th.text for th in row.find_elements(By.TAG_NAME, "th") if th.text]
    else:
        geo = cells[0].text.strip()
        if "Ontario" in geo:
            # Extract Ontario's quarterly population values
            ontario_data = [cell.text.replace(",", "") for cell in cells[1:]]
            break

# --------------------------------------------------------
# Construct DataFrame for Ontario Population
# --------------------------------------------------------
df = pd.DataFrame({
    "Quarter": headers[1:],  # Skip the first header ("Geography")
    "Population": ontario_data
})

# Convert population values to numeric format
df["Population"] = pd.to_numeric(df["Population"])

# --------------------------------------------------------
# Export to CSV
# --------------------------------------------------------
df.to_csv("ontario_population_2003_2023.csv", index=False)
print("✅ Ontario population data saved to 'ontario_population_2003_2023.csv'.")

# --------------------------------------------------------
# Close WebDriver
# --------------------------------------------------------
driver.quit()

In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd

def preprocess_population_data():
    file_path = "ontario_population.xlsx"
    output_path = "ontario_population_processed.csv"
    sheet_name = "ontario_population"

    # Load the Excel file
    xls = pd.ExcelFile(file_path)
    df = xls.parse(sheet_name)

    # Identify the row index where the actual data starts
    header_index = None
    for i, row in df.iterrows():
        if "Geography" in str(row[0]):  # Look for the header row containing "Geography"
            header_index = i
            break

    if header_index is None:
        raise ValueError("Could not find the header row containing 'Geography'")

    # Re-load the data with the correct header
    df_cleaned = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=header_index)

    # Drop any completely empty columns
    df_cleaned = df_cleaned.dropna(how='all', axis=1)

    # Reset column names to the first row and remove it from the data
    df_cleaned.columns = df_cleaned.iloc[0]
    df_cleaned = df_cleaned[1:].reset_index(drop=True)

    # Extract only the relevant row containing Ontario population data
    df_population = df_cleaned[df_cleaned.iloc[:, 0] == "Ontario"].set_index(df_cleaned.columns[0])

    # Transpose the data to make quarters a column
    df_melted = df_population.T.reset_index()
    df_melted.columns = ["Quarter", "Population"]

    # Remove any rows with missing population values
    df_melted["Population"] = df_melted["Population"].interpolate(method="linear")

    # Convert population values to integers
    df_melted["Population"] = df_melted["Population"].astype(int)

    # Save the processed data to CSV
    df_melted.to_csv(output_path, index=False)
    print(f"Processed data saved to {output_path}")

# Run the function
preprocess_population_data()

# Ontario Energy Demand Preprocess

In [ ]:
import pandas as pd

# --------------------------------------------------------
# Load Ontario Electricity Demand Dataset
# --------------------------------------------------------
df = pd.read_csv("ontario_electricity_demand.csv")  # Ensure this file is in the same directory or provide full path

# --------------------------------------------------------
# Check for Missing or Invalid Values
# --------------------------------------------------------
print("Missing values per column:")
print(df.isna().sum())

# --------------------------------------------------------
# Apply Sample-and-Hold Technique (Forward Fill)
# --------------------------------------------------------
df_cleaned = df.fillna(method='ffill')

# --------------------------------------------------------
# Preview Cleaned Data
# --------------------------------------------------------
print("\nCleaned Data Preview:")
display(df_cleaned.head())

# --------------------------------------------------------
# Save Cleaned Dataset to CSV
# --------------------------------------------------------
df_cleaned.to_csv("ontario_electricity_demand_cleaned.csv", index=False)
print("\n Cleaned dataset saved to 'ontario_electricity_demand_cleaned.csv'")


# Merge All Datasets

In [ ]:
import pandas as pd

# --------------------------------------------------------
# Step 1: Load Electricity Demand and Consolidated Datasets
# --------------------------------------------------------
# Load the dataset containing electricity demand and population data
df_demand = pd.read_csv("Ontario_Electricity_Demand_with_Population.csv")

# Load the consolidated dataset, which may include weather or processed station data
df_consolidated = pd.read_excel("Consolidated_Ontario_Final.xlsx")

# --------------------------------------------------------
# Step 2: Identify Common Columns for Merging
# --------------------------------------------------------
# Determine shared columns between the two datasets to use as join keys
common_columns = list(set(df_demand.columns) & set(df_consolidated.columns))
print("Common column(s) identified for merge:", common_columns)

# --------------------------------------------------------
# Step 3: Merge Datasets Based on Shared Column
# --------------------------------------------------------
if common_columns:
    # Perform an inner join on the first shared column
    merged_df = pd.merge(df_demand, df_consolidated, on=common_columns[0], how='inner')
    print(f"Datasets merged using key column: '{common_columns[0]}'")
    display(merged_df.head())

    # ----------------------------------------------------
    # Step 4: Sort Merged Data by Date Column
    # ----------------------------------------------------
    if 'Date' in merged_df.columns:
        # Convert the 'Date' column to datetime format
        merged_df['Date'] = pd.to_datetime(merged_df['Date'], errors='coerce')
        merged_df = merged_df.sort_values(by='Date')
        print("Merged data sorted by 'Date' column.")
        display(merged_df.head())
    else:
        print("No 'Date' column found in merged data. Sorting step skipped.")

    # ----------------------------------------------------
    # Step 5: Export Merged Dataset to CSV
    # ----------------------------------------------------
    merged_df.to_csv("Merged_Ontario_Energy_Dataset.csv", index=False)
    print("Merged dataset saved as 'Merged_Ontario_Energy_Dataset.csv'.")

else:
    print("No common columns found between the datasets. Merge operation skipped.")
